<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
# Библиотека warnings позволяет нам не выводить
# предупреждения в выводе сообщения.
import warnings
warnings.filterwarnings('ignore')

In [2]:
# параметры поключения
from personal import personal

In [74]:
connection = psycopg2.connect(
    dbname=personal['DBNAME'],
    user=personal['USER'],
    host=personal['HOST'],
    password=personal['PASSWORD'],
    port=personal['PORT']
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query = f'''
SELECT COUNT(DISTINCT v.id) /*функция подсчёта строк;значения столбца id*/
from vacancies v   /*из таблицы vacancies; присваиваем vacancies имя v*/
'''

In [5]:
# результат запроса
jobs = pd.read_sql_query(query, connection)
print(f'Всего {jobs.iloc[0,0]} вакансий.')


Всего 49197 вакансий.


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query = f'''
/*функция подсчёта строк;уникальные значения столбца id*/
SELECT COUNT(DISTINCT e.id) 
FROM employers e   /*из таблицы sql.vacancies*/
'''

In [7]:
# результат запроса
employers = pd.read_sql_query(query,connection)
print(f'Всего {employers.iloc[0,0]} работодатель.')

Всего 23501 работодатель.


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query = f'''
/*функция подсчёта строк;уникальные значения столбца id*/
SELECT COUNT(DISTINCT a.id) 
FROM areas a  /*из таблицы areas*/
'''

In [9]:
# результат запроса
towns = pd.read_sql_query(query,connection)
print(f'Всего {towns.iloc[0,0]} города.')

Всего 1362 города.


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query = f'''
SELECT 
    COUNT(DISTINCT i.id) /*счёт; уникальные значения столбца id*/
FROM industries i  /*из таблицы industries*/
'''

In [11]:
# результат запроса
industries = pd.read_sql_query(query,connection)
print(f'Всего {industries.iloc[0,0]} различных сфер деятельности.')

Всего 294 различных сфер деятельности.


## Выводы по предварительному анализу:


In [12]:
# выводы по предварительному анализу данных
print(
    '''
    - Всего рассматриваемых вакансий {0}. Хорошая база данных для проведения
    исследования.
    - Полный список всех зарегестрированных работодателей составляет {1}.
    Достаточный список, чтобы отследить спрос на рабочие места, уровень
    зарплаты, графика работы, требований в навыках и многое другое.
    - Всего {2} различных регионов(городов).
    - В рассматриваемой выборке {3} различных сферы деятельности.Это нам
    даёт возможность оценить множество сфер деятельности.
    '''.format(jobs.iloc[0, 0], employers.iloc[0, 0],
               towns.iloc[0, 0], industries.iloc[0, 0])
)


    - Всего рассматриваемых вакансий 49197. Хорошая база данных для проведения
    исследования.
    - Полный список всех зарегестрированных работодателей составляет 23501.
    Достаточный список, чтобы отследить спрос на рабочие места, уровень
    зарплаты, графика работы, требований в навыках и многое другое.
    - Всего 1362 различных регионов(городов).
    - В рассматриваемой выборке 294 различных сферы деятельности.Это нам
    даёт возможность оценить множество сфер деятельности.
    


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query = f'''
SELECT
    a.name area,  /*столбец name таблицы area*/
    COUNT(v.area_id) cnt  /*счёт;значения столбца area_id*/
FROM vacancies v  /*из таблицы industries*/
    JOIN areas a ON a.id = v.area_id  /*оператор соединения таблиц;
таблица matches; условие: id таблицы areas равен area_id таблицы vacancies*/
GROUP BY a.name  /*групируем таблицу по признаку a.name*/
ORDER BY 2 DESC  /*групирум по признаку */
'''

In [14]:
# результат запроса
job = pd.read_sql_query(query,connection)
job

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query = f'''
SELECT 
    COUNT(id)  /*счёт;значения столбца id*/
FROM 
    vacancies v  /*из таблицы vacancies*/
WHERE 
    /*фильтр salary_from не пустое или salary_to не пустое значение*/
    v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
'''

In [16]:
# результат запроса
df = pd.read_sql_query(query, connection)
salary_point = df.iloc[0, 0]
print('''
      В списке вакансий есть два уровня зарплат нижний "от" и верхний "до".
      У нас {0} вакансии, в которых есть запись хотя бы одного из двух полей.
      '''.format(salary_point)
      )


      В списке вакансий есть два уровня зарплат нижний "от" и верхний "до".
      У нас 24073 вакансии, в которых есть запись хотя бы одного из двух полей.
      


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query = f'''
SELECT 
    v.salary_from,  /*столбец name таблицы vacancies*/
    v.salary_to     /*столбец name таблицы vacancies*/
FROM 
    vacancies v     /*из таблицы vacancies*/
WHERE 
    /*фильтр salary_from не пустое или salary_to не пустое значение*/
    v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
'''

In [18]:
# результат запроса
df = pd.read_sql_query(query, connection)
salary_from = round(df['salary_from'].mean())
salary_to = round(df['salary_to'].mean())

print(f'Cредние значения для нижней границы зарплатной вилки {salary_from}.')
print(f'Cредние значения для верхней границы зарплатной вилки {salary_to}.')


Cредние значения для нижней границы зарплатной вилки 71065.
Cредние значения для верхней границы зарплатной вилки 110537.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query = f'''
SELECT 
    v.schedule,  /*столбец schedule таблицы vacancies*/
    lower(v.employment) employment,  /*lower() имя в нижний регистр */
    COUNT(v.id) quantity  /*счёт;значения столбца id*/
FROM 
    vacancies v  /*из таблицы vacancies*/
GROUP BY 1,2     /*группируе по 1-му далее по 2-му столбцу*/
ORDER BY 3 DESC  /*порядок по 3 столбцу; нисходящий*/
'''


In [20]:
# результат запроса
busy = pd.read_sql_query(query, connection)
busy


,schedule,employment,quantity
0,Полный день,полная занятость,35367
1,Удаленная работа,полная занятость,7802
2,Гибкий график,полная занятость,1593
3,Удаленная работа,частичная занятость,1312
4,Сменный график,полная занятость,940
5,Полный день,стажировка,569
6,Вахтовый метод,полная занятость,367
7,Полный день,частичная занятость,347
8,Гибкий график,частичная занятость,312
9,Полный день,проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query = f'''
SELECT 
    v.experience,  /*столбец experience таблицы vacancies*/
    COUNT(id) n_vacancies  /*счёт;значения столбца id*/
FROM 
    vacancies v  /*из таблицы vacancies*/
GROUP BY 1  /*группируем по 1-му столбцу*/
ORDER BY 2 ASC    /*порядок по 2-му столбцу; восходящий*/
'''


In [22]:
# результат запроса
df_experience = pd.read_sql_query(query, connection)
df_experience


,experience,n_vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


## Вывод по детальному анализу вакансий:

In [23]:
# выводы по детальному анализу вакансий
print('''
    1) Определили количество вакансий городам. В пятёрку лидеров со
    значениями входят:
    {0}: {1}, {2}: {3}, {4}: {5}, {6}: {7}, 
    {8}: {9}.

    2) У {10} вакансий указан уровень зарплаты, т.е около {11}% из общего 
    числа. Это зарплата, которую работадатель способен платить оставаясь,
    конкуретно способным.
    
    3) Средний уровень зарплаты по всем вакансиям находится в диапазоне
    от {12} до {13}. Этот показатель помогает понять, какой уровень зарплаты
    в среднем и на то как сильно она отличается от профессиональных умений
    сотрудника и его опыта работы.
    
    4) Мы определили наибольшую востребовнность по признаку "рабочего графика"
    и "типу занятости". На первой позиции стоит "{14}" и "{15}"
    с числом вакансий {16}, а на второй позиции "{17}" и 
    "{18}" с числом вакансий {19}. Можно предположить, что
    востребованость в удалённой работе растёт. С ростом технологий,
    в будущем этот показатель может только вырасти.
    
    5) Наиболее востребованный опыт работ у работодателей {20}.
    Это говорит о том что работодатель заинтересован взят сотрудника уже с
    опытом работы, чем без опыта, т.к. смогут больше заработать потратив
    меньше времени на воспитании, нового кадра. Или напротив нужен опытный
    сотрудник, который самостоятельно выполнит задачу и внесёт что-то новое.
'''.format(job.iloc[0][0], job.iloc[0][1], job.iloc[1][0], job.iloc[1][1],
           job.iloc[2][0], job.iloc[2][1], job.iloc[3][0], job.iloc[3][1],
           job.iloc[4][0], job.iloc[4][1], salary_point,
           round(salary_point/jobs.iloc[0][0]*100), salary_from, salary_to,
           busy.iloc[0][0], busy.iloc[0][1], busy.iloc[0][2], busy.iloc[1][0],
           busy.iloc[1][1], busy.iloc[1][2], df_experience.iloc[3][0])
      )



    1) Определили количество вакансий городам. В пятёрку лидеров со
    значениями входят:
    Москва: 5333, Санкт-Петербург: 2851, Минск: 2112, Новосибирск: 2006, 
    Алматы: 1892.

    2) У 24073 вакансий указан уровень зарплаты, т.е около 49% из общего 
    числа. Это зарплата, которую работадатель способен платить оставаясь,
    конкуретно способным.
    
    3) Средний уровень зарплаты по всем вакансиям находится в диапазоне
    от 71065 до 110537. Этот показатель помогает понять, какой уровень зарплаты
    в среднем и на то как сильно она отличается от профессиональных умений
    сотрудника и его опыта работы.
    
    4) Мы определили наибольшую востребовнность по признаку "рабочего графика"
    и "типу занятости". На первой позиции стоит "Полный день" и "полная занятость"
    с числом вакансий 35367, а на второй позиции "Удаленная работа" и 
    "полная занятость" с числом вакансий 7802. Можно предположить, что
    востребованость в удалённой работе растёт. С ростом технологи

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [75]:
# текст запроса
query = f'''
with x AS  /*строим таблицу x*/
((SELECT 
    e.name,  /*столбец name таблицы employers*/
    COUNT(v.id)  /*счёт;значения столбца id*/
FROM vacancies v  /*из таблицы vacancies*/
    /*JOIN внутренне объединение по employer_id таблицы vacancies и
    id таблицы employers */
    JOIN employers e ON v.employer_id = e.id
GROUP BY e.name  /*группируем таблицу по name*/
ORDER BY 2 DESC  /*порядок по 2-му столбцу; нисходящий*/
LIMIT 1)  /*выводим только одно значение*/
            
UNION all  /*оператор присоединения всех значений*/
            
(SELECT 
    e.name,      
    COUNT(v.id) 
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
GROUP BY e.name 
ORDER BY 2 DESC 
OFFSET 4   /*функция OFFSET исключает первые четыре строк*/
LIMIT 1))  /*выводим только одно значение*/


SELECT x.name /*столбец name таблицы x*/
FROM x  /*из таблицы x*/
'''


In [25]:
# результат запроса
df = pd.read_sql_query(query, connection)
employer = list(df['name'])
print('''
    Работодатель "{0}" находятся на первом и "{1}" на пятом месте по
    количеству вакансий.'''.format(employer[0], employer[1]))



    Работодатель "Яндекс" находятся на первом и "Газпром нефть" на пятом месте по
    количеству вакансий.


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
# текст запроса
query = f'''
SELECT 
    a.name town,  /*столбец name таблицы town*/
    COUNT(v.employer_id) qty_employers,  /*счёт;значения столбца employer_id*/
    COUNT(v.id) qty_vacancies,  /*счёт;значения столбца id*/
    COUNT(e.name) qty_employers_rigeon  /*счёт;значения столбца id*/
FROM areas a  /*из таблицы areas*/
    /*LEFT JOIN левое объединение по area_id таблицы vacancies и
    id таблицы employers */
    LEFT JOIN vacancies v ON v.area_id = a.id  
    JOIN employers e ON a.id = e.area 
    
WHERE v.id IS NULL  /*фильтр, где id пустое значение*/
GROUP BY a.name     /*группируем по имени*/
ORDER BY 4 DESC     /*порядок по 4-му столбцу; нисходящий*/
LIMIT 1             /*выводим только одно значение*/
'''


In [27]:
# результат запроса
df = pd.read_sql_query(query, connection)
employer_from = df.iloc[0, 0]
print(f'''
    В {employer_from} больше всего работодателей, но никто не указывает её,
    в каччестве места работы.
    ''')



    В Россия больше всего работодателей, но никто не указывает её,
    в каччестве места работы.
    


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества. Выведем первые 15 наиболее крупные компании. 

In [28]:
# текст запроса
query = f'''
SELECT 
    e.name,  /*столбец name таблицы employers*/
    COUNT(DISTINCT v.area_id)  /*счёт;уникальные значения столбца area_id*/
FROM vacancies v /*из таблицы vacancies*/
    /*JOIN внутреннее объединение по id таблицы employers и
    employer_id таблицы vacancies */
    JOIN employers e ON e.id = v.employer_id
GROUP BY e.id    /*группируем по id*/
ORDER BY 2 DESC  /*порядок по 2-му столбцу; нисходящий*/
LIMIT 15         /*выводим только 15 значений*/
'''


In [29]:
# результат запроса
employer_area_cnt = pd.read_sql_query(query, connection)
employer_area_cnt


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
# текст запроса
query = f'''
SELECT 
    COUNT(*)  /*счёт; число строк*/
FROM employers e  /*из таблицы employers*/
    /*LEFT JOIN левое объединение по id таблицы employers и
    employer_id таблицы employers_industries*/
    LEFT JOIN employers_industries e_i ON e.id = e_i.employer_id
WHERE e_i.industry_id IS NULL  /*фильтр, где industry_id пустое значение*/
'''


In [31]:
# результат запроса
empl_no_industry = pd.read_sql_query(query, connection)
print('''
    У {0} работодателей в анкете не указана сфера деятельности.'''
      .format(empl_no_industry.iloc[0, 0]))



    У 8419 работодателей в анкете не указана сфера деятельности.


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
# текст запроса
query = f'''
SELECT 
    e.name,                        /*столбец name таблицы town*/
    COUNT(e_i.industry_id)         /*счёт;значения столбца employer_id*/
FROM employers e
    /*LEFT JOIN левое объединение по признаку id таблицы employers и
    employer_id таблицы employers_industries */
    LEFT JOIN employers_industries e_i ON e.id = e_i.employer_id
    
WHERE e_i.industry_id IS NOT NULL  /*фильтр, где не пустое значение*/
GROUP BY e.id                      /*группируем id - идентификационный номер*/
HAVING COUNT(e_i.industry_id) = 4  /*фильтр, где счёт industry_id = 4*/
ORDER BY e.name ASC                /*порядок по  столбцу e.name; восходящий*/
    OFFSET 2                       /*OFFSET исключает первые две строк*/   
LIMIT 1                            /*выводим только одно значение*/
'''


In [33]:
# результат запроса
empl_yes_4_industry = pd.read_sql_query(query, connection)
empl_yes_4_industry
print('''
    Компания "{0}" находится на третьем месте среди тех, у кого указано
    четыре сферы деятельности.'''.format(empl_yes_4_industry.iloc[0, 0]))



    Компания "2ГИС" находится на третьем месте среди тех, у кого указано
    четыре сферы деятельности.


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
# текст запроса
query = f'''
SELECT 
    COUNT(e.id)  /*счёт;значения столбца id*/
FROM employers e
    /*объединение по ризнаку по employers_industries таблицы employer_id и
    id таблицы employers */
    LEFT JOIN employers_industries e_i ON e.id = e_i.employer_id
    /*объединение по ризнаку по id таблицы industries и
    id таблицы employers */
    JOIN industries i ON e_i.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''


In [35]:
# результат запроса
empl_developer = pd.read_sql_query(query, connection)
print('''
    {0} работодателей, указали «Разработку программного обеспечения»,
    в качестве сферы деятельности.'''.format(empl_developer.iloc[0, 0]))



    3553 работодателей, указали «Разработку программного обеспечения»,
    в качестве сферы деятельности.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
# Код для получения списка городов-милионников.
import requests
from bs4 import BeautifulSoup

# Определяем адрес страницы
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
# Выполняем GET-запрос
response = requests.get(url)
# Создаём объект BeautifulSoup, указывая html-парсер
page = BeautifulSoup(response.text, 'html.parser')

# В считаной странице page находим нужную таблицу по тегу <table>
table1 = page.find('table', class_='standard')

# В table1 найдём название колонки таблицы по тегу <th></th>,
# и занесём все значения которые будут в промежутке тега <th></th>
# в список headers
headers = []
for i in table1.find_all('th'):
    title = i.text  # .text т.е. только имя колонки занесём в title
    headers.append(title)  # добавим имя колонки в список

# Создаём DataFrame и передаём список headers в качестве названий колонок.
mydata = pd.DataFrame(columns=headers)

# Заносим данные в DataFrame с помощью цикла
#
# Список значений начинаем со второго значения [1:], т.к. в нём далее
# не будет тега <td></td>, иначе в row будет пустой
# список, который не может быть занесён в DataFrame в качестве строки
# данных из пяти значений, т.к. пять колонок. Надо помнить что первая
# строка данных table1 содержит названия колонок, которые мы
# применили и создали df см.выше.
for j in table1.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    # Определяем высоту таблицы и номер следущей строки.
    length = len(mydata)
    # Заносим список (из 5 значений) в строку DataFrame,
    # по номеру строки Length.
    mydata.loc[length] = row

# Изменим имя Воронеж[a] на Воронеж, иначе запрос не найдёт этот город
mydata.loc[13]['Город'] = 'Воронеж'

# В переменную city_millions запишем список городов в виде кортежа
city_millions = tuple(mydata["Город"])
print(city_millions)


('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [37]:
# текст запроса
query = f'''
WITH x AS
(SELECT 
    a.name town,                          /*покажем столбец name*/
    COUNT(v.id)::integer cnt              /*счёт;значения столбца id*/
FROM vacancies v                          /*из таблицы vacancies*/ 
JOIN employers e ON v.employer_id = e.id  /*объединяем с таблице employers*/
JOIN areas a ON v.area_id = a.id          /*объединяем с таблице areas*/
WHERE e.name='Яндекс'                     /*фильтр значений*/ 
    /*city_millions используем как список значений перданных в Metabase*/
    AND (a.name in {city_millions} OR a.name = 'Воронеж')
GROUP BY a.name                           /*группируем по имени*/
ORDER BY 1 ASC)                           /*порядок восходящий*/

SELECT*
FROM x

union all                              /*объединяем со строкой по вертикали*/               

(SELECT 
    'Total',SUM(x.cnt)     
FROM x)  
'''


In [38]:
# результат запроса
df = pd.read_sql_query(query,connection)
print(df)

               town  cnt
0         Волгоград   24
1           Воронеж   32
2      Екатеринбург   39
3            Казань   25
4         Краснодар   30
5        Красноярск   23
6            Москва   54
7   Нижний Новгород   36
8       Новосибирск   35
9              Омск   21
10            Пермь   25
11   Ростов-на-Дону   25
12           Самара   26
13  Санкт-Петербург   42
14              Уфа   26
15        Челябинск   22
16            Total  485


## Вывод по анализу работодателей:

In [39]:
# выводы по анализу работодателей
print('''
    1) На первом месте по количеству вакансий занимает "{0}",на пятом
    "{1}" это говорит о том, что наиболее крупные компании, размещают
    больше вакансий.
    
    2) Наибольшее количество работодателей нахдятся в {2}. В качестве
    региона для работы никто не указывает её. Это действительно так, т.к.
    {3}  большая страна с большой территорией. Работодатели стараются
    найти сотрудников, в близи тех районов, где осуществляют деятельность,
    для удобства взаимодействия.
    
    3) Компания "{4}" ведёт поиск сотрудников в {5} регионе и является
    лидером по данному показателю.
    
    4) {6} работодателей не указана сферы деятельности, т.е. {7}% из
    общего числа. Причиной возможно служит то, что для компании это
    новое направление и сфера деятельности ещё не сформировалась,
    либо работа требует от работника высоких профессиональных навыков 
    в различных областях.
    
    5) Компания {8} находится на третьем месте, у которой указано
    четыре сферы деятельности. Возможно такие компании имеют различные
    способы зароботка, поэтому имею различные сферы деятельности,
    а также то что все вместе они дополняют, друг друга и приносят
    компании дополнительную прибыль.
    
    6) У {9} работодатей в качестве сферы деятельности указана
    «Разработка программного обеспечения». Это составляет около  
    {10}% от общего числа компаний. Это весомая доля организаций
    и с каждым годом этот показатель может расти.
    Для сайта в качестве рекомендации:
    Так как IT профессия весьма сложная то для кандидатов можно было
    бы устроить микро-тесты по ключевым навыкам. Таким образом соискатель
    мог бы проверить свои силы, а работодатель сэконимил время при подборе,
    нужного работника, до того как провести собеседование.
    
    7) Для компании «Яндекс» мы вывели список регионов-миллионников,
    в которых представлены вакансии компании, вместе с количеством
    вакансий в этих регионах. Как результат во всех городах милионниках
    есть вакансии этой компании.
    '''
      .format(employer[0], employer[1], employer_from, employer_from,
              employer_area_cnt.iloc[0][0], employer_area_cnt.iloc[0][1],
              empl_no_industry.iloc[0, 0],
              round((empl_no_industry.iloc[0, 0]/employers.iloc[0, 0])*100),
              empl_yes_4_industry.iloc[0, 0], empl_developer.iloc[0, 0],
              round((empl_developer.iloc[0, 0]/employers.iloc[0, 0])*100))
      )



    1) На первом месте по количеству вакансий занимает "Яндекс",на пятом
    "Газпром нефть" это говорит о том, что наиболее крупные компании, размещают
    больше вакансий.
    
    2) Наибольшее количество работодателей нахдятся в Россия. В качестве
    региона для работы никто не указывает её. Это действительно так, т.к.
    Россия  большая страна с большой территорией. Работодатели стараются
    найти сотрудников, в близи тех районов, где осуществляют деятельность,
    для удобства взаимодействия.
    
    3) Компания "Яндекс" ведёт поиск сотрудников в 181 регионе и является
    лидером по данному показателю.
    
    4) 8419 работодателей не указана сферы деятельности, т.е. 36% из
    общего числа. Причиной возможно служит то, что для компании это
    новое направление и сфера деятельности ещё не сформировалась,
    либо работа требует от работника высоких профессиональных навыков 
    в различных областях.
    
    5) Компания 2ГИС находится на третьем месте, у которой указано
 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [70]:
# текст запроса
query = f'''
SELECT 
    COUNT(DISTINCT v.id)     /* счёт уникальных;значения столбца id*/
FROM 
    vacancies v              /* из таблицы areas*/
    
/* Фильтр по работе с данными. Различные комбинации вводов пользователей*/    
WHERE v.name LIKE '%данн%'   
    OR v.name LIKE '%data%'
    OR v.name LIKE '%Data%'
    OR v.name LIKE '%DATA%' 
    OR v.name LIKE '%Данн%'
    OR v.name LIKE '%ДАНН%'
        '''

In [71]:
# результат запроса
df = pd.read_sql_query(query,connection)
v_data = df.iloc[0]['count'] 
print(f'{v_data} вакансия имеет отношение к данным.')

1771 вакансия имеет отношение к данным.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [42]:
# текст запроса
query = f'''
SELECT COUNT(DISTINCT v.id)  /* счёт;уникальные значения id*/
    FROM vacancies v         /* из таблицы vacancies*/
/* Фильтр по специальности дата саентиста.*/
WHERE (v.name ILIKE '%data scientist%' 
    OR v.name ILIKE '%data science%' 
    OR v.name ILIKE '%исследователь данных%'
    OR name ILIKE '%machine learning%'
    OR v.name ILIKE '%машинн%обучен%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%'))
    
    AND
    
    /* Фильтр по специальности дата саентиста, без опыта работы.*/
    (v.name ILIKE '%junior%' OR
    v.experience ILIKE '%Нет опыта%' OR
    v.employment ILIKE '%Стажировка%')
    '''


In [43]:
# результат запроса
df = pd.read_sql_query(query, connection)
v_data_bigner = df.iloc[0]['count']
print(f'{v_data_bigner} подходящих вакансий для начинающего дата-сайентиста')


51 подходящих вакансий для начинающего дата-сайентиста


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
# текст запроса
query = f'''
SELECT
    count(v.name)   /* покажем столбец name */

FROM vacancies v    /* из таблицы vacancies*/

WHERE       /* фильтр по специальности дата саентиста*/
    (
    name ILIKE '%data scientist%' 
    OR name ILIKE '%data science%' 
    OR name ILIKE '%исследователь данных%'
    OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%'
    AND key_skills IS NOT NULL
    )
    
    AND 
    /* фильтр по навыку знаний SQL и postgres*/   
    (key_skills LIKE '%SQL%' OR key_skills ILIKE'%postgres%')
'''

In [45]:
# результат запроса
df = pd.read_sql_query(query, connection)
v_data_SQL = df.iloc[0]['count']
print('В {0} вакансии для DS в качестве ключа указан навык SQL или postgres.'
      .format(v_data_SQL))


В 201 вакансии для DS в качестве ключа указан навык SQL или postgres.


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [46]:
# текст запроса
query = f'''
SELECT
    COUNT(v.id)     /* счёт;значения столбца id*/

FROM vacancies v    /* из таблицы vacancies*/

/* фильтр по специальности дата саентиста*/
WHERE (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%' 
    OR name ILIKE '%исследователь данных%'
    OR ((name LIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL))
    
    /* фильтр по навыку знаний Python*/
    AND key_skills LIKE '%Python%'
    '''

In [47]:
# результат запроса
df = pd.read_sql_query(query, connection)
v_data_Python = df.iloc[0]['count']
print('В {0} вакансии для DS в качестве ключа указан навык Python.'
      .format(v_data_Python))


В 351 вакансии для DS в качестве ключа указан навык Python.


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [48]:
# текст запроса
query = f'''
SELECT
    /* Мы определяем сколько навыков указано в строке. Для этого от общей
    длины строки отнимаем туже длину, но убрав символ CHR(9) (т.е. 'TAB').
    'TAB' служит разделителем между навыками. Окрыгляем среднее значение.*/
    ROUND(AVG((LENGTH(v.key_skills) 
    - LENGTH(replace(v.key_skills,CHR(9),'')))::integer + 1),2) as result
FROM vacancies v    /* из таблицы vacancies*/

/* фильтр по специальности дата саентиста*/ 
WHERE (v.name ILIKE '%data scientist%' 
    OR v.name ILIKE '%data science%' 
    OR v.name ILIKE '%исследователь данных%' 
    OR v.name ILIKE '%машинн%обучен%' 
    OR v.name ILIKE '%machine learning%' 
    OR (v.name LIKE '%ML%'  AND v.name NOT ILIKE '%HTML%'))
    
    /* фильтр, где ячейки key_skills не пустые*/            
    AND v.key_skills IS NOT NULL
'''


In [49]:
# результат запроса
df = pd.read_sql_query(query, connection)
v_skills = df.iloc[0][0]
print('В среднем указываю около {0} навыков для вакансии DS.'
      .format(v_skills))


В среднем указываю около 6.41 навыков для вакансии DS.


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [76]:
# текст запроса
query = f'''with x as (
SELECT 
    v.experience,   /* покжем столбец experience */
    v.salary_from,  /* покжем столбец salary_from */
    v.salary_to,    /* salary_to */
    /* Складываем min и max зарплату и делим на 2, чтобы найти среднюю.*/
    (v.salary_from + v.salary_to)/2 as result
FROM vacancies v
/* фильтр по специальности дата саентиста*/ 
WHERE (v.name ILIKE '%data scientist%' OR
        v.name ILIKE '%data science%' OR
        v.name ILIKE '%исследователь данных%' OR
        v.name ILIKE '%машинн%обучен%' OR
        v.name ILIKE '%machine learning%' OR
        v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
        /* фильтр, где указана min и max зарплата(ЗП) */      
        AND (v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL)
                
UNION  ALL  /* объединение таблиц по вертикали, всех строк со всеми*/ 
    
SELECT 
    v.experience,   
    v.salary_from,
    v.salary_to,
    /* coalesce(v.salary_from,0) если не указана зарплата заменит на 0*/
    coalesce(v.salary_from,0) + coalesce(v.salary_to,0) 
FROM vacancies v
WHERE (v.name ILIKE '%data scientist%' OR
        v.name ILIKE '%data science%' OR
        v.name ILIKE '%исследователь данных%' OR
        v.name ILIKE '%машинн%обучен%' OR
        v.name ILIKE '%machine learning%' OR
        v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') 
        /* фильтр, где указана либо min или max зарплата(ЗП) */       
        AND ((v.salary_from IS NULL AND v.salary_to IS NOT NULL)
        OR (v.salary_from IS NOT NULL AND v.salary_to IS NULL)))
    
/*experience::text переводит все данные столбца в текст*/ 
/*AVG(result)::integer переводит все данные в целый тип и выводит средний
результат*/   
SELECT experience::text, AVG(result)::integer median
FROM x
GROUP BY experience /*групируем по опыту*/ 
    
UNION All  /* объединение таблиц по вертикали, всех строк со всеми*/

/* Приводим таблицы к одному типу, чтобы их объединить.*/    
SELECT 'median_salary'::text, AVG(result)::integer
FROM x
'''

In [77]:
# результат запроса
df = pd.read_sql_query(query,connection)
df.set_index('experience',inplace=True)
s_median = int(df.loc['median_salary','median'])

print (f'''Средняя зарплата DS составляет:\n {df}''')
# 

Средняя зарплата DS составляет:
                     median
experience                
От 3 до 6 лет       243115
От 1 года до 3 лет  139675
Нет опыта            74643
median_salary       177379


## Выводы по предметному анализу:

In [52]:
# выводы по предметному анализу
print(''' 
    1) {0} вакансий имеет отношение к данным.
    2) {1} подходящая вакансий для начинающего дата-сайентиста.
    3) {2} вакансия для DS, в которых в качестве ключевого навыка указан 
    SQL или postgres.
    4) {3} вакансия, в которых в качестве ключевого навыка указан Python.
    5) {4} ключевых навыков в среднем указывают работодатели в вакансиях
    для DS.
    6) На зарплату в {5}  в среднем может рассчитывать дата-сайентист.
    '''.format(v_data, v_data_bigner, v_data_SQL, v_data_Python,
               v_skills, s_median)
      )


 
    1) 1771 вакансий имеет отношение к данным.
    2) 51 подходящая вакансий для начинающего дата-сайентиста.
    3) 201 вакансия для DS, в которых в качестве ключевого навыка указан 
    SQL или postgres.
    4) 351 вакансия, в которых в качестве ключевого навыка указан Python.
    5) 6.41 ключевых навыков в среднем указывают работодатели в вакансиях
    для DS.
    6) На зарплату в 177379  в среднем может рассчитывать дата-сайентист.
    


# Общий вывод по проекту

In [53]:
# подведем итог исследования, обобщите выводы
print(f''' 
      Закончив проект можно сделать общие выводы:
      1.Предварительный анализ показал,что наша БД состоит
      - из 50тыс. вакасий заявленых от работодателей;
      - всего около 23,5тыс. компаний в БД в поиске работников;
      - в выбрке около 1,3тыс. регионов(городов);
      - и около 300 различных сфер деятельности компаний.
      
      2.Детальный анализ вакансиий показал, что:
      - больше всего вакансии в крупных городах;
      - у половина предложений о работе указан размер оплаты труда;
      - средний уровень зарплаты в варьируется от 70 до 110 тыс.;
      - самый распространённые типы занятости и графика(полный день
      полная занятость и второй удалённая работа полная занятость);
      - самые востребованые специалисты от 1 года до 3 лет.
      
      3.Анализ работодателей показал,что:
      - больше всего вакансий размещают крупные компании;
      - большинство работодателей из России;
      - наиболее крупные компании ведут поиск работников в большем
      количестве городов;
      - 36% компаний не имеет определённой сферы деятельности;
      - встречаются компании максимум с 4-мя сферами деятельности;
      - 15% компаний занимаются "Разработкой программного обспечения";
      - крупные компания публикуют вакансии в самых крупных городах,
      пример "Яндекс"
      
      4.Предметный анализ показал, что:
      - около 1800 вакансии имеют отношения к данным;
      - около 50 вакансий для начинающего дата-саентиста;
      - Python и SQL 2 наиболее важных навыка дата-саентиста;
      - в среднем дата-саентисту нужно обладать 6 ключевыми навыками;
      - зарплата дата-саентиста в среднем около 180тыс. и зависит от опыта.
      ''')

 
      Закончив проект можно сделать общие выводы:
      1.Предварительный анализ показал,что наша БД состоит
      - из 50тыс. вакасий заявленых от работодателей;
      - всего около 23,5тыс. компаний в БД в поиске работников;
      - в выбрке около 1,3тыс. регионов(городов);
      - и около 300 различных сфер деятельности компаний.
      
      2.Детальный анализ вакансиий показал, что:
      - больше всего вакансии в крупных городах;
      - у половина предложений о работе указан размер оплаты труда;
      - средний уровень зарплаты в варьируется от 70 до 110 тыс.;
      - самый распространённые типы занятости и графика(полный день
      полная занятость и второй удалённая работа полная занятость);
      - самые востребованые специалисты от 1 года до 3 лет.
      
      3.Анализ работодателей показал,что:
      - больше всего вакансий размещают крупные компании;
      - большинство работодателей из России;
      - наиболее крупные компании ведут поиск работников в большем
      кол

## Дополнение.

В качестве дополнения определим какие ещё ключевые навыки важны для дата саентиста.

Для этого сделаем:
1. Напишим запрос, который вернёт нам таблицу *df_skills* с одной колонкой *skills*(навыки). Это навыки для каждой вакансий на профессию дата-саентиста.

In [54]:
# текст запроса
query = f'''
SELECT 
    /* Заменим символ CHR(9) на $ для наглядности.*/
    replace(v.key_skills,CHR(9),'$') skills
FROM vacancies v  /* из таблицы vacancies*/
WHERE 
    /* фильтр по специальности дата саентиста*/ 
    (v.name ILIKE '%data scientist%' OR v.name ILIKE '%data science%' 
    OR v.name ILIKE '%исследователь данных%'
    OR name ILIKE '%machine learning%'
    OR v.name ILIKE '%машинн%обучен%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')) 
    '''


In [55]:
# результат запроса
df_skills = pd.read_sql_query(query,connection)

# Количество рассматриваемых вакансий для дата-саентиста
ds_vacancies = df_skills.size

# вывод результат запроса
# Навыкы разделены символом $.
df_skills

,skills
0,Python$SQL$Работа в команде$OpenCV$ML
1,Oracle Pl/SQL$Базы данных$Machine Learning$Маш...
2,Python$OpenCV$Tensorflow$Computer Vision$Нейро...
3,Python$Flask$Linux$AWS$Git
4,Python$SQL$NLP$Scikit-learn$Machine Learning
...,...
475,Python$Numpy$PyTorch$Pandas$Математический ана...
476,Python$Scikit-learn$Pandas$Git$Numpy
477,Qt$ООП$C++$MS Visual Studio$Разработка компьют...
478,SCALA$SQL$Математический анализ$Статистический...


2. Из полученой таблицы *df_skills* выберим только уникальные навыки.

In [56]:
# Заносим в список skillsDS уникальные названия навыков считаных
# из таблицы df_skills.
def extract_skill(df):
    """Функция перебирает все навыки и сохраняет уникальные.

    Args:
        df (_DataFrame_): Таблица навыков от разных работодателей 
                          и вакансий
    """
    # Создаём пустой список lib
    lib = []

    # Цикл перебирает каждую строку таблицы df.
    for i in range(len(df)):
        # В переменную skills считываем навыки одной вакансии.
        skills = str(df.loc[i]['skills'])
        # Разбиваем skills на навыки и сохраняем в виде списка.
        skills = skills.split("$")
        # В цикле перебираем навыки в списке.
        for skill in skills:
            # Если навык встречается в первый раз сохраняем его в
            # skillsDS и наоборот.
            if skill.lower() not in lib:
                if len(skill) > 1: # Фильтр от умений в одну букву
                    # Для фильтра от повторяемости заносим всписок навык
                    # в нижнем регистре.
                    lib.append(skill.lower())
                    # Заносит навык в список в том виде котором он существует.
                    skillsDS.append(skill)


In [57]:
# Создаём пустой список скилов дата-саентиста.
skillsDS = []

#Функция заполнить список skillsDS уникальными навыками.
extract_skill(df_skills)

3. Напишим запрос который будет проверять в скольких вакансиях встречается навык. Все полученые значения занесём в нашу таблицу *table_skillsDS* с колонками *skill*(навык) и cnt(частота встречаемости).

In [58]:
# Функция определения частоту встречаимости навыка в вакансиях.
def find_cnt_skill(skill):
    """ Определяет как часто встречается навык в вакансии.

    Args:
        skill (string): Передаём название навыка.

    Returns:
         numpy.int64 : Возвращает частоту встречаимости навыка в вакансиях.
    """
    # Запрос
    query = f'''
    SELECT 
        count(*)
    FROM vacancies v
    WHERE 
        (v.name ILIKE '%data scientist%' OR v.name ILIKE '%data science%' 
        OR v.name ILIKE '%исследователь данных%'
        OR name ILIKE '%machine learning%'
        OR v.name ILIKE '%машинн%обучен%'
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')) 
        
        AND v.key_skills ILIKE '%{skill}%'               
        '''
    # результат запроса
    cnt = pd.read_sql_query(query, connection)
    # Переводим в целое число и возвращаем как частоту встречаемости.
    return int(cnt.iloc[0][0])


In [59]:
# Создаём пустой DataFrame с колонками skill(навык) и
# cnt(как часто навык встречается в заявке от работодателя)
table_skillsDS = pd.DataFrame(columns=['skill', 'cnt'])

# Заполняем нашу таблицу skills_DSt по строчно.
# Для этого перебираем все навыки из skillsDS
for skill in skillsDS:
    # Опредляем длинну таблицы.
    length = len(table_skillsDS)
    # Передаём название навыка в функцию.
    count = find_cnt_skill(skill)
    # Заносим в таблицу навык и частоту встречаимости.
    table_skillsDS.loc[length] = [skill, count]


In [60]:
# Удалим столбец индекса.
table_skillsDS.set_index('skill', inplace=True)

# Выведем топ 15 навыков в профессии DSt,
# и как часто они встречаются.
table_skillsDS = table_skillsDS.sort_values(by='cnt', ascending=False)
table_skillsDS.head(25)

,cnt
skill,
Python,351
SQL,201
Machine Learning,141
IT,104
ML,74
Git,70
Статистика,66
Data science,64
Математическая статистика,62


В таблице выше, есть позиции схожие по смыслу, объединим их:
- Machine Learning, ML и Машинное обучение, Deep Learning;
- Статистика, Математическая статистика;
- Анализ данных, Математический анализ, Data Analysis, Статистический анализ;
- Аналитическое мышление, Аналитические исследования;

In [61]:
# Функция объединения навыков
def sum_skills(lst):
    """Суммирует навыки между собой, оставив один и удалив лишние.

    Args:
        lst (_type_): _description_

    Returns:
        _type_: _description_
    """
    result = table_skillsDS.at[lst[0], 'cnt']
    for skill in lst[1:]:
        result += table_skillsDS.at[skill, 'cnt']
        table_skillsDS.drop(skill, axis=0, inplace=True)
    return result


In [62]:
# Суммируем ячейки
ml = ['Машинное обучение','Machine Learning','ML','Deep Learning']
st = ['Статистика', 'Математическая статистика'] 
analysis = ['Анализ данных', 'Математический анализ',
      'Data Analysis', 'Статистический анализ']
analytics = ['Аналитическое мышление', 'Аналитические исследования']

In [63]:
# Суммируем ячейки и удаляем лишнее.
table_skillsDS.at['Машинное обучение','cnt'] = sum_skills(ml)
table_skillsDS.at['Статистика','cnt'] = sum_skills(st)
table_skillsDS.at['Анализ данных','cnt'] = sum_skills(analysis)
table_skillsDS.at['Аналитическое мышление','cnt'] = sum_skills(analytics)

# Приеобразуем данные в "int32"
table_skillsDS['cnt'] = table_skillsDS['cnt'].astype('int32')

In [64]:
# Результат 
table_skillsDS = table_skillsDS.sort_values(by='cnt',ascending=False)
table_skillsDS.head(15)

,cnt
skill,
Python,351
Машинное обучение,271
SQL,201
Анализ данных,169
Статистика,128
IT,104
Git,70
Data science,64
PyTorch,61


In [65]:
# Импортируе библиотеки для визуализации.
import plotly
import plotly.express as px

In [66]:

df = table_skillsDS.copy().head(10)

# Построим график.
# Отобразим на графике Топ-10 навыков и их долю в % из общего числа
# профессий "ds_vacancies".
fig = px.bar(
    data_frame=df, #датафрейм
    x=df.index, #ось x
    y=df["cnt"], #ось y
    color= df.index, #расцветка в зависимости от навыка
    text = round(df["cnt"]/ds_vacancies*100,1), #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=1000, #ширина
    title='Top 10 skills for Data Scientist in percentage(%).' #заголовок
    
)

#отображаем его
fig.show()

## Итоги дополнительного исследования:

In [67]:
print('''
    1. К основным навыкам профессии DS можем отнести не только "Python" и
    "SQL", а также:
    - "Машинное обучение";
    - "Анализ данных";
    - "Статистика";
    - "Git", "Pandas", "NLP" и другие.
   
    2. На построенном графике видно как часто встречается навык,в профессии
    датасаентиста в процентном соотношении(%) к общиму числу вакансий(480).
    Пример: "Статистика" в целом важна в специальности DS имеет 26.7%.
   
    3. Это лишь часть исследований, которые были разобраны в данном проекте. 
    В качестве ключевых направление для сайта HeadHunter могу предложить,
    для удобства пользованием сайтом как для соискателей так и работодателей:
        - провести исследования и определить группы профессий;
        - сгрупировать навыки для различных профессий, т.к. сказать
        стандартизировать их при заполнении; Это упростит заполнения,
        работодателями необходимых ячеек, а соискателям даст возможность
        понимание,что от них хотят;
        - добавить для соискателей возможные мини тесты для проверки их
        навыков; это даст возможность проверить себя, а работодателям
        с экономить время при поиске кандидатов на должность и другое.
    ''')
    


    1. К основным навыкам профессии DS можем отнести не только "Python" и
    "SQL", а также:
    - "Машинное обучение";
    - "Анализ данных";
    - "Статистика";
    - "Git", "Pandas", "NLP" и другие.
   
    2. На построенном графике видно как часто встречается навык,в профессии
    датасаентиста в процентном соотношении(%) к общиму числу вакансий(480).
    Пример: "Статистика" в целом важна в специальности DS имеет 26.7%.
   
    3. Это лишь часть исследований, которые были разобраны в данном проекте. 
    В качестве ключевых направление для сайта HeadHunter могу предложить,
    для удобства пользованием сайтом как для соискателей так и работодателей:
        - провести исследования и определить группы профессий;
        - сгрупировать навыки для различных профессий, т.к. сказать
        стандартизировать их при заполнении; Это упростит заполнения,
        работодателями необходимых ячеек, а соискателям даст возможность
        понимание,что от них хотят;
        - добавить для со